In [25]:
from sklearn.model_selection import train_test_split
import pandas as pd
import shutil
import os
import json

In [4]:
namelist = os.listdir('./dirty_mnist/')
answer = pd.read_csv('dirty_mnist_answer.csv')

In [15]:
train, val = train_test_split(namelist, test_size=0.2, random_state=23)
train_num = [int(t[:5]) for t in train]
val_num = [int(v[:5]) for v in val]
train_ohe = [True if i in train_num else False for i in range(50000)]
val_ohe = [True if i in val_num else False for i in range(50000)]
train_ans = answer[train_ohe]
val_ans = answer[val_ohe]

In [24]:
os.makedirs('./dirty_mnist_train', exist_ok=True)
os.makedirs('./dirty_mnist_val', exist_ok=True)
for t in train:
    shutil.move('./dirty_mnist/' + t, './dirty_mnist_train')
for v in val:
    shutil.move('./dirty_mnist/' + v, './dirty_mnist_val')

In [52]:
def csv_to_coco(df, json_name):
    dic = {
        "info": {
            "description": "DACON DIRTY MNIST DATASET",
            "url": "https://oranz.tistory.com/",
            "version": "1.0",
            "year": 2021,
            "contributor": "DACON",
            "date_created": "2021/02/18"
        },
        "licenses": [{
            "url": "http://creativecommons.org/licenses/by-nc-sa/2.0/",
            "id": 1,
            "name": "Attribution-NonCommercial-ShareAlike License"
        }],
        "images": [],
        "annotations": [],
        "categories": []
    }
    
    for i in range(1, 27):
        dic["categories"].append({"supercategory": "alphabet", "id": i, "name": chr(i + 96)})
        
    anno_id = 1
    for r in range(len(df)):
        image_id = int(df.iloc[r, 0]) # cast to int for JSON serialization
        name = str(image_id)
        name = '0' * (5 - len(name)) + name + '.png'
        dic["images"].append({
            "license": 1,
            "file_name": name,
            "coco_url": "",
            "height": 256,
            "width": 256,
            "date_captured": "2020-05-19 23:03:57",
            "flickr_url": "",
            "id": image_id
        })
        
        for c in range(1, 27):
            if df.iloc[r, c] == 1:
                dic["annotations"].append({
                    "segmentation": [],
                    "area": 232,
                    "iscrowd": 0,
                    "image_id": image_id,
                    "bbox": [1, 5, 4, 9],
                    "category_id": c,
                    "id": anno_id
                })
                anno_id += 1
    with open(json_name, "w") as json_file:
        json.dump(dic, json_file)

In [53]:
csv_to_coco(val_ans, "val.json")
csv_to_coco(train_ans, "train.json")